#  Limpieza de Datos - SaaS Revenue Analytics

Este notebook contiene el proceso de exploración y limpieza de datos para un proyecto de análisis financiero de una empresa SaaS (Software as a Service).

## 📁 Datasets Utilizados

Se trabajó con 4 archivos CSV:

- `users.csv`: Información básica de usuarios (país, fecha de registro).
- `subscriptions.csv`: Detalles del plan contratado, precio mensual y duración.
- `payments.csv`: Historial de pagos mensuales realizados.
- `churn.csv`: Cancelaciones de suscripciones y sus razones.

## ✅ Pasos realizados

- Revisión y conversión de tipos de datos (fechas, numéricos, strings).
- Validación de llaves y referencias entre tablas (UserID).
- Detección y tratamiento de fechas inválidas.
- Identificación de usuarios sin pagos o sin suscripciones.
- Eliminación de registros inconsistentes.
- Guardado de datasets limpios en la carpeta `data/`.

> Esta limpieza permite construir una tabla de hechos robusta para análisis posteriores usando PySpark.


In [1]:
import pandas as pd

# Cargar los datos
df_users = pd.read_csv("../data/users.csv")
df_subs = pd.read_csv("../data/subscriptions.csv")
df_payments = pd.read_csv("../data/payments.csv")
df_churn = pd.read_csv("../data/churn.csv")

# Mostrar primeras filas
display(df_users.head())
display(df_subs.head())
display(df_payments.head())
display(df_churn.head())


,UserID,Country,RegistrationDate
0,U0001,India,2022-10-25
1,U0002,Brazil,2022-01-18
2,U0003,India,2022-10-17
3,U0004,Spain,2022-05-17
4,U0005,USA,2022-03-01


,UserID,Plan,DurationMonths,MonthlyPrice,StartDate
0,U0001,Premium,12,30,2022-01-14
1,U0002,Standard,3,20,2022-08-25
2,U0003,Standard,1,20,2022-10-08
3,U0004,Basic,3,10,2022-08-27
4,U0005,Basic,12,10,2022-03-05


,UserID,PaymentDate,Amount,PaymentMethod
0,U0001,2022-01-14,30,PayPal
1,U0001,2022-02-14,30,Credit Card
2,U0001,2022-03-14,30,Credit Card
3,U0001,2022-04-14,30,Bank Transfer
4,U0001,2022-05-14,30,PayPal


,UserID,ChurnDate,ChurnReason
0,U0002,2022-09-30,Not useful
1,U0009,2023-01-03,Too expensive
2,U0010,2022-05-18,Too expensive
3,U0011,2022-09-07,Other
4,U0012,2022-07-12,Other


In [2]:
# Revisar valores nulos en todos los datasets

print("USERS")
print(df_users.isnull().sum())
print("\n----------------------\n")

print("SUBSCRIPTIONS")
print(df_subs.isnull().sum())
print("\n----------------------\n")

print("PAYMENTS")
print(df_payments.isnull().sum())
print("\n----------------------\n")

print("CHURN")
print(df_churn.isnull().sum())


USERS
UserID              0
Country             0
RegistrationDate    0
dtype: int64

----------------------

SUBSCRIPTIONS
UserID            0
Plan              0
DurationMonths    0
MonthlyPrice      0
StartDate         0
dtype: int64

----------------------

PAYMENTS
UserID           0
PaymentDate      0
Amount           0
PaymentMethod    0
dtype: int64

----------------------

CHURN
UserID         0
ChurnDate      0
ChurnReason    0
dtype: int64


In [7]:

print("USERS")
print(df_users.info())
print("\n----------------------\n")


USERS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   UserID            500 non-null    object
 1   Country           500 non-null    object
 2   RegistrationDate  500 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB
None

----------------------



In [4]:

print("SUBSCRIPTIONS")
print(df_subs.info())
print("\n----------------------\n")


SUBSCRIPTIONS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   UserID          500 non-null    object
 1   Plan            500 non-null    object
 2   DurationMonths  500 non-null    int64 
 3   MonthlyPrice    500 non-null    int64 
 4   StartDate       500 non-null    object
dtypes: int64(2), object(3)
memory usage: 19.7+ KB
None

----------------------



In [5]:
print("PAYMENTS")
print(df_payments.info())
print("\n----------------------\n")

PAYMENTS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860 entries, 0 to 2859
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserID         2860 non-null   object
 1   PaymentDate    2860 non-null   object
 2   Amount         2860 non-null   int64 
 3   PaymentMethod  2860 non-null   object
dtypes: int64(1), object(3)
memory usage: 89.5+ KB
None

----------------------



In [6]:
print("CHURN")
print(df_churn.info())

CHURN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   UserID       112 non-null    object
 1   ChurnDate    112 non-null    object
 2   ChurnReason  112 non-null    object
dtypes: object(3)
memory usage: 2.8+ KB
None


In [8]:
# Convertir columnas de fecha a tipo datetime
df_users["RegistrationDate"] = pd.to_datetime(df_users["RegistrationDate"], errors="coerce")
df_subs["StartDate"] = pd.to_datetime(df_subs["StartDate"], errors="coerce")
df_payments["PaymentDate"] = pd.to_datetime(df_payments["PaymentDate"], errors="coerce")
df_churn["ChurnDate"] = pd.to_datetime(df_churn["ChurnDate"], errors="coerce")

# Verificamos los tipos de datos
print(df_users.dtypes)
print(df_subs.dtypes)
print(df_payments.dtypes)
print(df_churn.dtypes)


UserID                      object
Country                     object
RegistrationDate    datetime64[ns]
dtype: object
UserID                    object
Plan                      object
DurationMonths             int64
MonthlyPrice               int64
StartDate         datetime64[ns]
dtype: object
UserID                   object
PaymentDate      datetime64[ns]
Amount                    int64
PaymentMethod            object
dtype: object
UserID                 object
ChurnDate      datetime64[ns]
ChurnReason            object
dtype: object


In [9]:
# USERS: duplicados por UserID
dup_users = df_users.duplicated(subset='UserID').sum()
print(f"Duplicados en users.csv: {dup_users}")

# PAYMENTS: duplicados por UserID + PaymentDate
dup_payments = df_payments.duplicated(subset=['UserID', 'PaymentDate']).sum()
print(f"Duplicados en payments.csv: {dup_payments}")

# SUBSCRIPTIONS: duplicados por UserID
dup_subs = df_subs.duplicated(subset='UserID').sum()
print(f"Duplicados en subscriptions.csv: {dup_subs}")


Duplicados en users.csv: 0
Duplicados en payments.csv: 0
Duplicados en subscriptions.csv: 0


In [10]:
df_subs["MonthlyPrice"] = df_subs["MonthlyPrice"].astype(float)
df_payments["Amount"] = df_payments["Amount"].astype(float)


In [11]:
df_subs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   UserID          500 non-null    object        
 1   Plan            500 non-null    object        
 2   DurationMonths  500 non-null    int64         
 3   MonthlyPrice    500 non-null    float64       
 4   StartDate       500 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 19.7+ KB


In [12]:
df_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860 entries, 0 to 2859
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   UserID         2860 non-null   object        
 1   PaymentDate    2860 non-null   datetime64[ns]
 2   Amount         2860 non-null   float64       
 3   PaymentMethod  2860 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 89.5+ KB


In [13]:
# Convertimos todos los IDs a string por seguridad
df_users['UserID'] = df_users['UserID'].astype(str)
df_subs['UserID'] = df_subs['UserID'].astype(str)
df_payments['UserID'] = df_payments['UserID'].astype(str)
df_churn['UserID'] = df_churn['UserID'].astype(str)

# Subscriptions sin usuario en users
subs_huerfanos = df_subs[~df_subs['UserID'].isin(df_users['UserID'])]
print(f"Subscriptions sin usuario asociado: {len(subs_huerfanos)}")

# Payments sin usuario en users
payments_huerfanos = df_payments[~df_payments['UserID'].isin(df_users['UserID'])]
print(f"Payments sin usuario asociado: {len(payments_huerfanos)}")

# Churn sin usuario en users
churn_huerfanos = df_churn[~df_churn['UserID'].isin(df_users['UserID'])]
print(f"Churn sin usuario asociado: {len(churn_huerfanos)}")


Subscriptions sin usuario asociado: 0
Payments sin usuario asociado: 0
Churn sin usuario asociado: 0


In [14]:
# Asegurarse de que los IDs estén como string
df_users['UserID'] = df_users['UserID'].astype(str)
df_subs['UserID'] = df_subs['UserID'].astype(str)

# Unir users con subscriptions por UserID
merged_df = pd.merge(df_subs, df_users, on='UserID', how='left')

# Filtrar registros donde StartDate sea anterior a la fecha de registro
fechas_invalidas = merged_df[merged_df['StartDate'] < merged_df['RegistrationDate']]

# Mostrar resultados
print(f"Usuarios con StartDate anterior a RegistrationDate: {len(fechas_invalidas)}")
print(fechas_invalidas[['UserID', 'RegistrationDate', 'StartDate']])


Usuarios con StartDate anterior a RegistrationDate: 249
    UserID RegistrationDate  StartDate
0    U0001       2022-10-25 2022-01-14
2    U0003       2022-10-17 2022-10-08
6    U0007       2022-12-09 2022-08-29
7    U0008       2022-03-23 2022-01-02
8    U0009       2022-11-03 2022-01-04
..     ...              ...        ...
493  U0494       2022-07-30 2022-03-24
496  U0497       2022-07-03 2022-02-10
497  U0498       2022-05-18 2022-01-31
498  U0499       2022-03-21 2022-01-28
499  U0500       2022-05-10 2022-04-26

[249 rows x 3 columns]


In [17]:
# Crear una copia para no alterar el original de inmediato
df_users_corr = df_users.copy()

# Unimos la fecha de inicio de suscripción por usuario
df_users_corr = df_users_corr.merge(
    df_subs[['UserID', 'StartDate']],
    on='UserID',
    how='left',
    suffixes=('', '')
)

# Corregimos la RegistrationDate si StartDate es menor
df_users_corr['RegistrationDate'] = df_users_corr[['RegistrationDate', 'StartDate']].min(axis=1)

# Eliminamos columna auxiliar
df_users_corr.drop(columns='StartDate', inplace=True)

# Reemplazamos el dataframe original por el corregido
df_users = df_users_corr


In [ ]:
# Asegurarse de que los IDs estén como string
df_users['UserID'] = df_users['UserID'].astype(str)
df_subs['UserID'] = df_subs['UserID'].astype(str)

# Unir users con subscriptions por UserID
merged_df = pd.merge(df_subs, df_users, on='UserID', how='left')

# Filtrar registros donde StartDate sea anterior a la fecha de registro
fechas_invalidas = merged_df[merged_df['PaymentMethod'] < merged_df['StartDate']]

# Mostrar resultados
print(f"Usuarios con StartDate anterior a RegistrationDate: {len(fechas_invalidas)}")
print(fechas_invalidas[['UserID', 'RegistrationDate', 'StartDate']])


Usuarios con StartDate anterior a RegistrationDate: 0
Empty DataFrame
Columns: [UserID, RegistrationDate, StartDate]
Index: []


In [19]:
# Aseguramos que UserID esté como string en ambos
df_payments['UserID'] = df_payments['UserID'].astype(str)
df_subs['UserID'] = df_subs['UserID'].astype(str)

# Unimos pagos con suscripciones por UserID
pagos_con_fecha = pd.merge(df_payments, df_subs[['UserID', 'StartDate']], on='UserID', how='left')

# Buscamos pagos realizados antes de la fecha de inicio del plan
pagos_antes_de_inicio = pagos_con_fecha[pagos_con_fecha['PaymentDate'] < pagos_con_fecha['StartDate']]

# Mostramos resultados
print(f"Pagos realizados antes del StartDate: {len(pagos_antes_de_inicio)}")
print(pagos_antes_de_inicio[['UserID', 'PaymentDate', 'StartDate']].head())


Pagos realizados antes del StartDate: 0
Empty DataFrame
Columns: [UserID, PaymentDate, StartDate]
Index: []


In [20]:
# Asegurar que UserID esté en string
df_churn['UserID'] = df_churn['UserID'].astype(str)
df_subs['UserID'] = df_subs['UserID'].astype(str)

# Unimos churn con suscripciones para obtener el StartDate
churn_check = pd.merge(df_churn, df_subs[['UserID', 'StartDate']], on='UserID', how='left')

# Detectar casos donde se canceló antes de haber empezado
churn_invalido = churn_check[churn_check['ChurnDate'] < churn_check['StartDate']]

# Mostrar resultado
print(f"Cancelaciones antes del inicio del plan: {len(churn_invalido)}")
print(churn_invalido[['UserID', 'StartDate', 'ChurnDate']].head())


Cancelaciones antes del inicio del plan: 36
   UserID  StartDate  ChurnDate
2   U0010 2022-10-07 2022-05-18
4   U0012 2022-08-26 2022-07-12
5   U0017 2022-10-30 2022-09-30
7   U0028 2022-07-03 2022-06-16
10  U0051 2022-08-11 2022-06-04


In [21]:
# Paso 1: Obtener último PaymentDate por usuario
ultimo_pago = df_payments.groupby('UserID')['PaymentDate'].max().reset_index()
ultimo_pago.columns = ['UserID', 'UltimoPago']

# Paso 2: Unir churn con StartDate
df_churn_corr = df_churn.merge(df_subs[['UserID', 'StartDate']], on='UserID', how='left')

# Paso 3: Unir también con último pago
df_churn_corr = df_churn_corr.merge(ultimo_pago, on='UserID', how='left')

# Paso 4: Marcar usuarios inválidos (Churn antes de Start y sin pagos)
churn_invalidos = df_churn_corr[
    (df_churn_corr['ChurnDate'] < df_churn_corr['StartDate']) &
    (df_churn_corr['UltimoPago'].isnull())
]

# Mostrar cuántos se eliminarán
print(f"Registros a eliminar por inconsistencia y sin pagos: {len(churn_invalidos)}")

# Paso 5: Filtrar solo válidos
df_churn_corr = df_churn_corr[~df_churn_corr['UserID'].isin(churn_invalidos['UserID'])]

# Paso 6: Corregir ChurnDate con el último pago si aplica
df_churn_corr['ChurnDate'] = df_churn_corr.apply(
    lambda row: row['UltimoPago'] if row['ChurnDate'] < row['StartDate'] else row['ChurnDate'],
    axis=1
)

# Paso 7: Limpiar columnas auxiliares
df_churn_corr.drop(columns=['StartDate', 'UltimoPago'], inplace=True)

# Paso 8: Reemplazar el original
df_churn = df_churn_corr.reset_index(drop=True)


Registros a eliminar por inconsistencia y sin pagos: 0


In [22]:
# Asegurar que UserID esté en string
df_churn['UserID'] = df_churn['UserID'].astype(str)
df_subs['UserID'] = df_subs['UserID'].astype(str)

# Unimos churn con suscripciones para obtener el StartDate
churn_check = pd.merge(df_churn, df_subs[['UserID', 'StartDate']], on='UserID', how='left')

# Detectar casos donde se canceló antes de haber empezado
churn_invalido = churn_check[churn_check['ChurnDate'] < churn_check['StartDate']]

# Mostrar resultado
print(f"Cancelaciones antes del inicio del plan: {len(churn_invalido)}")
print(churn_invalido[['UserID', 'StartDate', 'ChurnDate']].head())

Cancelaciones antes del inicio del plan: 0
Empty DataFrame
Columns: [UserID, StartDate, ChurnDate]
Index: []


In [23]:
# Guardar los CSV limpios
df_users.to_csv('../data/users_clean.csv', index=False)
df_subs.to_csv('../data/subscriptions_clean.csv', index=False)
df_payments.to_csv('../data/payments_clean.csv', index=False)
df_churn.to_csv('../data/churn_clean.csv', index=False)
